In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# OSCAR MOVIES
At this section, we going to explore the dataset of Oscar Movies. Before define the task, lets we what the data set contain about and what can we analyze. 

In [ ]:
#import module
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
%matplotlib inline

In [ ]:
movies = pd.read_csv('../input/oscar-movies/oscars_df.csv')
movies.shape

In [ ]:
#movies dataseata overview
movies.head()


The movie contain 571 rows and 30 columns. Lets see the columns attributes. It seem contain NaN values that we need to clean.

In [ ]:
movies.info()

In [ ]:
movies.nunique()

the Dataset column cointain 8 float, 3 integer, and 19 objects. The total rows suppose to be 571 but some data seem has null value. We will start to fix the null value in order to clean the data. In this study, we are going to include only Oscar Year, Film, Award, IMDB Rating, IMDB Votes, TOmatometer Rating, Tomatometer Count, Audience Rating and Audience Count.

In [ ]:
oscars = movies[['Oscar Year','Film','Award','IMDB Rating','IMDB Votes','Tomatometer Rating','Tomatometer Count','Audience Rating','Audience Count']]
oscars.head()

In [ ]:
oscars.isnull().sum()

In [ ]:
#duplicate values
oscars.duplicated().sum()

We found that 29 columns contain null values. Dataset does not show duplicate values. We going to see how many percentage of data did not countain value. if the missing data over 15% of total data, the data will be dropped.

In [ ]:
total_null = oscars.isnull().sum().sort_values(ascending=False) 
count = oscars.count().sort_values(ascending=False) 
movies_p = (((oscars.isnull().sum()/oscars.isnull().count()).sort_values(ascending=False)).round(2))*100 #Get the Percentage
movies_p.head(30)

We found that data missing each row over 23%. It would be good idea to drop the null values

In [ ]:
oscars1= oscars.dropna()
oscars1['IMDB Votes'] = oscars1['IMDB Votes'].str.replace(',','')
oscars1['IMDB Votes'] = oscars1['IMDB Votes'].astype(float)
oscars1.info()


As we can see data rows reduced from 571 to 439 rows. the IMDB Votes converted into float. Since we have three indicator rating (IMDB, Tomatometer, and Audiance), we are going to add column repsenting it's average value from three. Please in mind that while scale of IMDB from 1 to 10, the other two using scale 1 to 100. 

In [ ]:
# average rating
oscars1['avg.rating']=((oscars1['IMDB Rating']*oscars1['IMDB Votes']+oscars1['Tomatometer Rating']*0.1*oscars1['Tomatometer Count']+oscars1['Audience Rating']*0.1*oscars1['Audience Count'])/(oscars1['IMDB Votes']+oscars1['Tomatometer Count']+oscars1['Audience Count'])).round(2)
oscars1.head(10)

On the Next step, we will see how sthe rating trend for oscar winning movies.

In [ ]:
# Graphic of Oscar Winner Movies
winner_rate =oscars1[oscars1.Award =='Winner'].sort_values('Oscar Year',ascending=True)
plt.figure(figsize=(20,6))
lw = 2
plt.plot(winner_rate['Oscar Year'],winner_rate['avg.rating'],  color='red', lw=lw, label='Average Rating')
plt.plot(winner_rate['Oscar Year'],winner_rate['IMDB Rating'],  color='blue', lw=lw, label='IMDB Rating')
plt.plot(winner_rate['Oscar Year'],winner_rate['Tomatometer Rating']/10,  color='green', lw=lw, label='Tomatometer Rating')
plt.plot(winner_rate['Oscar Year'],winner_rate['Audience Rating']/10,  color='yellow', lw=lw, label='Audience Rating')
plt.xlabel('DATE')
plt.ylabel('Rating')
plt.xticks(rotation=45, fontsize=8)
plt.title('Figure 1 : Rating of Oscar Winner Movies')
plt.legend(loc="lower right")
plt.grid()
plt.show()

According to Graphic above, Oscar Winner Movies in beginning of the time show have low rating. By its time, the ratings are improved. from three source of rating system (IMDB, Tomatometer, Audiance), it show similiar trends. The average rating will be used for further analysis. most oscar winning movies are in range of above 7.
We going to compare its rating to nominee movies ratings.

In [ ]:
# omparation of nominee rating and winner rating 
nomination_rate =oscars1[oscars1.Award =='Nominee'].sort_values('Oscar Year',ascending=True)
w1= winner_rate[winner_rate['Oscar Year'] != '1927/28']
w1= w1[winner_rate['Oscar Year'] != '1930/31']
plt.figure(figsize=(20,6))
lw = 2
plt.plot(nomination_rate['Oscar Year'],nomination_rate['avg.rating'],  color='black', lw=lw,label='Nominee')
plt.plot(w1['Oscar Year'],w1['avg.rating'],  color='red', lw=lw,label='Oscar Winner')

plt.xlabel('DATE')
plt.ylabel('Rating')
plt.xticks(rotation=45, fontsize=8)
plt.title('Figure 2: Rating of Movies')
plt.legend(loc="lower right")
plt.grid()
plt.show()

The Oscar winner movies ratings show close but slightly above nominee movies. Lets see if top 20 high rate movie contain nominee movies as wells.

In [ ]:
# most high rate movie
high_rating =oscars1.sort_values('avg.rating',ascending=False)

In [ ]:
winner_nominee = high_rating[high_rating['avg.rating']> 8].groupby('Award').agg({'avg.rating': 'count'}).sort_values('avg.rating',ascending = False).reset_index()
plt.figure(figsize=(10,6))
# Plot the pie chart
plt.pie(winner_nominee['avg.rating'],labels=winner_nominee['Award'], autopct='%1.1f%%', startangle=30)
# Equal aspect ratio ensures that pie is drawn as a circle.
plt.axis('equal')
plt.subplots_adjust(wspace=2, hspace=1)
plt.title("Figure 3: Winner and Nominee for movie Rating above 8")
# Display the graph onto the screen
plt.show()

According to Figure 3 above, we can see that from movie with rating above 8, only   24 % of oscar winning movie on the list while 76% are nominee. 

In [ ]:
# TOP 10 Oscar Winner Movie
top10awards=winner_rate.sort_values('avg.rating',ascending=False).head(10)     
plt.figure(figsize=(10,6))
clr =("blue", "forestgreen", "gold", "red", "purple",'cadetblue','hotpink','orange','darksalmon','brown')
plt.bar(top10awards['Film'], top10awards['avg.rating'], color=clr)
plt.xlabel('Film')
plt.ylabel('Ratings')
plt.xticks(rotation=70, fontsize=10)
plt.title('Figure 4: TOP 10 High Ratings Oscar Movies')
plt.grid()
plt.show()

The Top 3 high rating oscar winning movies are "the Godfather", followed by " Schindler's List" and Forest Gump.

In [ ]:
### TOP 10 Oscar High Rating Movie
top10rating=high_rating.sort_values('avg.rating',ascending=False).head(10)  
plt.figure(figsize=(10,6))
clr =("blue", "forestgreen", "gold", "red", "purple",'cadetblue','hotpink','orange','darksalmon','brown')
plt.bar(top10rating['Film'], top10rating['avg.rating'], color=clr)
plt.xlabel('Film')
plt.ylabel('Ratings')
plt.xticks(rotation=80, fontsize=10)
plt.title('Figure 5: TOP 10 High Ratings Movies')
plt.grid()
plt.show()

If we look up the general, The top 3 rating movies are "The Shawshank Redemption", "The Godfather", and "Pulp Fiction". If we compare Figure 4 and Figure 5, We found that 6 movies are not winning the oscars but having high ratings while only 6 oscar movie include in the top 10 list.
However, since the rating score between of movies very tight, we can see that in general, nominee and winner are considered high rated movies that can be translated as movies that liked by most viewers.